<a href="https://colab.research.google.com/github/IlyaZelinskiy/Wav2vec_CFT_SHIFT_LAB/blob/main/Custom_trainer_emb_emotion_wav2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets==1.18.3
!pip install transformers==4.17.0
!pip install eng-to-ipa
!pip install jiwer 

import eng_to_ipa as p
from datasets import load_dataset, load_metric
metric = load_metric("accuracy")

In [ ]:
from datasets import load_dataset


slue = load_dataset("asapp/slue", "voxceleb")

Downloading:   0%|          | 0.00/12.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.55G [00:00<?, ?B/s]

/root/.cache/huggingface/datasets/downloads/extracted/50bec784754ee4fb0f60e54e86b1f664529fdacf86e46524cf962ff59bf36f75/slue-voxceleb


0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset slue downloaded and prepared to /root/.cache/huggingface/datasets/asapp___slue/voxceleb/2.4.0/6eef0a9b9ce2693a4f0ffdf4dd74ab5dc6e95bcd07366ac2a2b275a5b296f90d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
slue

DatasetDict({
    train: Dataset({
        features: ['id', 'audio', 'speaker_id', 'normalized_text', 'sentiment', 'start_second', 'end_second'],
        num_rows: 5777
    })
    validation: Dataset({
        features: ['id', 'audio', 'speaker_id', 'normalized_text', 'sentiment', 'start_second', 'end_second'],
        num_rows: 1454
    })
    test: Dataset({
        features: ['id', 'audio', 'speaker_id', 'normalized_text', 'sentiment', 'start_second', 'end_second'],
        num_rows: 3553
    })
})

In [ ]:
slue = slue.remove_columns(['id', 'speaker_id', 'start_second','start_second', 'end_second'])

In [ ]:
slue_train = slue['train'].add_column(name="label", column=arr)

In [ ]:
slue_test = slue['validation'].add_column(name="label", column=arr1)

In [ ]:
arr = []
for i in range(len(slue['train']['sentiment'])):
    if slue['train']['sentiment'][i] == 'Negative':
       arr.append(0)
    elif slue['train']['sentiment'][i] == 'Neutral':
        arr.append(1)
    elif slue['train']['sentiment'][i] == 'Positive':
        arr.append(2)

for i in range(48):
    arr.append(3)
len(arr)

5777

In [ ]:
arr1 = []
for i in range(len(slue['validation']['sentiment'])):
    if slue['validation']['sentiment'][i] == 'Negative':
       arr1.append(0)
    elif slue['validation']['sentiment'][i] == 'Neutral':
        arr1.append(1)
    elif slue['validation']['sentiment'][i] == 'Positive':
        arr1.append(2)

for i in range(17):
    arr1.append(3)
len(arr1)

1454

In [ ]:
slue = slue_train.remove_columns(["id",  "speaker_id", "start_second", "end_second"])
slue = slue_test.remove_columns(["id",  "speaker_id", "start_second", "end_second"])

In [ ]:
vocab = {
     'Negative' : 0,
     'Neutral' : 1,
     'Positive' : 2
}

In [ ]:
label2id = {
    'Negative': 0,
 'Neutral': 1,
 'Positive': 2,
 'unknown' : 3

}

In [ ]:
id2label = {
    0: 'Negative',
 1: 'Neutral',
 2: 'Positive',
 3: 'unknown'
}

In [ ]:
from transformers import AutoFeatureExtractor
model_checkpoint = "facebook/wav2vec2-base"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
feature_extractor

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/configuration_utils.py:356: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [ ]:
max_duration = 1.0  # seconds


In [ ]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=feature_extractor.sampling_rate, 
        max_length=int(feature_extractor.sampling_rate * max_duration), 
        truncation=True, 
    )
    return inputs

In [ ]:
encoded_dataset_train = slue_train.map(preprocess_function, remove_columns=["audio"], batched=True)
encoded_dataset_test = slue_test.map(preprocess_function, remove_columns=["audio"], batched=True)


  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

NameError: ignored

In [ ]:
encoded_dataset_train.remove_columns(["normalized_text",  "sentiment", 'speaker_id','id','start_second','end_second'])

Dataset({
    features: ['label', 'input_values'],
    num_rows: 5777
})

In [ ]:
encoded_dataset_test.remove_columns(["normalized_text",  "sentiment", 'speaker_id','id','start_second','end_second'])

Dataset({
    features: ['label', 'input_values'],
    num_rows: 1454
})

In [ ]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_checkpoint, 
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)
model.to('cuda')

In [ ]:
model_name = model_checkpoint.split("/")[-1]
batch_size = 4
args = TrainingArguments(
    f"{model_name}-finetuned-ks",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    print(predictions)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset_train,
    eval_dataset=encoded_dataset_test,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()


The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForSequenceClassification.forward` and have been ignored: sentiment, id, speaker_id, end_second, start_second, normalized_text. If sentiment, id, speaker_id, end_second, start_second, normalized_text are not expected by `Wav2Vec2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 5777
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 225


Epoch,Training Loss,Validation Loss


In [ ]:
encoded_dataset_test['sentiment'][3]

'Neutral'

In [ ]:
encoded_dataset_test['input_values'][0]

In [ ]:
import torch
input_values = torch.tensor(encoded_dataset_test['input_values'][3], device="cuda").unsqueeze(0)

In [ ]:
logits = model(input_values).logits

In [ ]:
predictions = torch.argmax(logits, axis=1)

In [ ]:
predictions[0]

tensor(1, device='cuda:0')